In [ ]:
# 1. Imports

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
# 2. Data loading

In [ ]:
ratings = pd.read_csv('../data/ratings.csv')

In [ ]:
# 3. Prepare Surprise dataset

In [ ]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(ratings[['userid', 'movieid', 'rating']], reader)

In [ ]:
# 4. Train-test split

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# 5. SVD training

In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:
# 6. Evaluation

In [ ]:
predictions = algo.test(testset)
print('Test RMSE:', accuracy.rmse(predictions))

In [ ]:
# 7. Recommend for a demo user

In [ ]:
def recommend_for_user(algo, user_id, movies, ratings, n=5):
    seen_movieids = set(ratings[ratings.userid == user_id].movieid)
    all_movieids = set(movies.movieid)
    unseen = list(all_movieids - seen_movieids)
    preds = [(mid, algo.predict(user_id, mid).est) for mid in unseen]
    topn = sorted(preds, key=lambda x: x[1], reverse=True)[:n]
    titles = movies.set_index('movieid').loc[[i[0] for i in topn]].title
    return titles.reset_index(drop=True)

In [ ]:
demo_user_id = ratings.userid.sample(1).iloc[0]
print(recommend_for_user(algo, demo_user_id, movies, ratings))